In [1]:
import pandas as pd
import os
import psycopg2
from sqlalchemy import create_engine


In [16]:
beer_types = pd.read_csv("Beer_Styles_All_Groups_BB.csv")
beer_types.head()

,beer_style,subgroup_5
0,American Barleywine,Dark Ale
1,American Black Ale,Dark Ale
2,American Brown Ale,Dark Ale
3,American Dark Wheat Ale,Dark Ale
4,American Double / Imperial Stout,Dark Ale


In [17]:
df = pd.read_csv("beer_reviews.csv")
parsed_data = df.drop_duplicates()
parsed_data = parsed_data.dropna()
parsed_data = parsed_data.reset_index()
parsed_data.head()

,index,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [27]:
parsed_data = parsed_data.loc[parsed_data["beer_style"] != "Specialty"]
parsed_data = parsed_data.loc[parsed_data["beer_style"] != "Sour Ale"]
parsed_data.beer_style.unique()

array(['Pale Ale', 'Dark Ale', 'Pale Lager', 'Dark Lager'], dtype=object)

In [19]:
for name in beer_types.index:
    parsed_data = parsed_data.replace(to_replace=beer_types.iloc[name]['beer_style'], value=beer_types.loc[name]['subgroup_5'])

In [6]:
parsed_data.head()


,index,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Pale Ale,1.5,1.5,Sausa Weizen,5.0,47986
1,1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,Pale Ale,3.0,3.0,Red Moon,6.2,48213
2,2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Dark Ale,3.0,3.0,Black Horse Black Beer,6.5,48215
3,3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,Pale Lager,2.5,3.0,Sausa Pils,5.0,47969
4,4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,Pale Ale,4.0,4.5,Cauldron DIPA,7.7,64883


In [7]:
print(parsed_data.columns)

Index(['index', 'brewery_id', 'brewery_name', 'review_time', 'review_overall',
       'review_aroma', 'review_appearance', 'review_profilename', 'beer_style',
       'review_palate', 'review_taste', 'beer_name', 'beer_abv',
       'beer_beerid'],
      dtype='object')


In [20]:
review = parsed_data[["index", "review_profilename", "review_overall","review_aroma", "review_appearance", "review_palate", "review_taste", "beer_beerid"]]
review.head()

,index,review_profilename,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_beerid
0,0,stcules,1.5,2.0,2.5,1.5,1.5,47986
1,1,stcules,3.0,2.5,3.0,3.0,3.0,48213
2,2,stcules,3.0,2.5,3.0,3.0,3.0,48215
3,3,stcules,3.0,3.0,3.5,2.5,3.0,47969
4,4,johnmichaelsen,4.0,4.5,4.0,4.0,4.5,64883


In [21]:
brewery = parsed_data[["brewery_id", "brewery_name"]]
brewery = brewery.drop_duplicates()
brewery.head()

,brewery_id,brewery_name
0,10325,Vecchio Birraio
4,1075,Caldera Brewing Company
10,163,Amstel Brouwerij B. V.
392,1454,Broad Ripple Brew Pub
402,850,Moon River Brewing Company


In [22]:
users = parsed_data[["review_profilename"]]
users = users.drop_duplicates()
users = users.reset_index(drop=True)
users = users.reset_index(drop=False)
users = users.rename(columns={'index': 'reviewer_id'})
users.head()

,reviewer_id,review_profilename
0,0,stcules
1,1,johnmichaelsen
2,2,oline73
3,3,Reidrover
4,4,alpinebryant


In [28]:
beer_types = parsed_data[["beer_style"]]
beer_types = beer_types.drop_duplicates()
beer_types = beer_types.reset_index(drop=True)
beer_types = beer_types.reset_index(drop=False)
beer_types = beer_types.rename(columns={'index': 'beer_style_id'})
beer_types.head()

,beer_style_id,beer_style
0,0,Pale Ale
1,1,Dark Ale
2,2,Pale Lager
3,3,Dark Lager


In [29]:
beer_types.beer_style.unique()


array(['Pale Ale', 'Dark Ale', 'Pale Lager', 'Dark Lager'], dtype=object)

In [30]:


beers = parsed_data[["beer_beerid", "beer_name", "beer_style", "beer_abv", "brewery_id"]]
beers = beers.drop_duplicates()
beers.head()

,beer_beerid,beer_name,beer_style,beer_abv,brewery_id
0,47986,Sausa Weizen,Pale Ale,5.0,10325
1,48213,Red Moon,Pale Ale,6.2,10325
2,48215,Black Horse Black Beer,Dark Ale,6.5,10325
3,47969,Sausa Pils,Pale Lager,5.0,10325
4,64883,Cauldron DIPA,Pale Ale,7.7,1075


In [31]:
for name in beer_types.index:
    beers = beers.replace(to_replace=beer_types.iloc[name]['beer_style'], value=beer_types.loc[name]['beer_style_id'])


In [32]:
rds_connection_string = "root:WE0ew4F3H2U0@finalproject.cqyw28rarsf4.us-east-1.rds.amazonaws.com:5432/postgres"
engine = create_engine(f'postgresql://{rds_connection_string}')

brewery.to_sql(name='brewery', con=engine, if_exists='append', index=False)
beer_types.to_sql(name='beer_style', con=engine, if_exists='append', index=False)
beers.to_sql(name='beer', con=engine, if_exists='append', index=False)
users.to_sql(name='reviewers', con=engine, if_exists='append', index=False)